# Objetivos del notebook
---
En este notebook vamos a explorar algunas de las ***best practices*** de las librerías **pandas** y **sklearn**.

**[pandas](https://pandas.pydata.org)** es una librería enfocada al análisis y manipulación de datos tabulares.<br>
**[sklearn](https://scikit-learn.org/stable/)** a su vez es una librería enfocada al machine learning. Contiene una colección de algoritmos de entrenamiento así como objetos para el preprocesamiento de datos y validación de modelos de ML.

Estos dos paquetes junto con [numpy](https://numpy.org), [matplotlib](https://matplotlib.org), [xgboost](https://xgboost.readthedocs.io/en/stable/) constituyen las principales librerías del ecosistema Data Science/Machine Learning en Python.

El notebook busca fomentar el trabajo en clase. Por este motivo, a lo largo del notebook se irán proponiendo diferentes preguntas y casos prácticos.

Al final del notebook, el alumno se tiene que sentir cómodo con los siguientes funcionalidades de pandas y sklearn.

---
## pandas
En el apartado de pandas queremos dar a conocer algunas de las funciones "ocultas" de pandas y que pueden ser muy útiles a la hora de manipular un dataset. En concreto, buscamos:

1. Entender en que consiste el **pandas groupby** y como podemos utilizarlo para hacer cálculo de manera **rápida, limpia y eficiente.**
2. Aprender a realizar **varios cálculos a la vez sobre diferentes columnas (CORE IDEA)** de un df todo dento del pandas groupby.
3. Entender el método de **pandas transform (CORE IDEA)**.
4. Definir y utilizar **funciones propias (custom functions)** dentro de un groupby.
5. Ver como podemos utilizar las **funciones propias** para hacer un cálculo dentro de un groupby utilizando **A LA VEZ** la información de 2 columnas diferentes (o más).
---

## sklearn
Scikit-learn es una de las librerías **CORE** de Machine Learning. Su impacto es tan grande, que muchas de las librerías nuevas que aparecen en el ecosistema, "imitan" la filosofía, el diseño y siguen los patrones de código marcados por sklearn. Por este motivo vamos a dar a conocer algunas de sus funcionalidades más avanzadas y que puede ayudar al alumno a la hora de: **procesar sus datos, prevenir "data leakage" en los modelos, "concatenar" varias operaciones de procesamineto con pipelines, optimizar los hiperparametros de los modelos así como disponer de nuevas técnicas para evaluar los modelos.**

Buscamos que el alumno al final de la sesión se sienta cómodo con:

1. Saber utilizar los **Transformers (CORE IDEA)** de sklearn.
2. Concatenar diferentes Transformers con un **Pipeline (CORE IDEA)**.
3. Aprender a utilizar el **ColumnTransformer (CORE IDEA)** para poder realizar diferentes transformaciones sobre diferentes columnas.
4. Aprender diferentes técnicas de **Cross Validation** disponibles dentro de sklearn.
5. Poder utilizar el **GridSearch** o **RandomizedGridSearch** para optimizar los hiperparametros de un modelo de Machine Learning.
6. **Saber optimizar los hiperparametros de un Pipeline de Machine Learning.**
---

El presente notebook contiene muchas funcionalidades avanzadas que puede requerir varias lecturas para su comprensión.
Pensamos que las secciones marcadas con **CORE IDEA** son las que mayor beneficio le reportarán al alumno. Por este motivo, le animamos a que dedique especial atención a estas ideas.

Al final del notebook, hay un sección de referencias y lecturas recomendables para que el alumno pueda seguir profundizando en estos conceptos.

---

## Índice
<a id='index'></a>

[Imports del notebook](#imports_notebook)<br>


## Best practices pandas
[Helpers pandas](#helpers_pandas)<br>
[¿Que es el pandas groupby?](#pandas_groupby)<br>
[Pandas transform (**CORE IDEA**)](#pandas_transform)<br>
[Groupby eficiente (**CORE IDEA**)](#pandas_gbe)<br>
[Funciones personalizadas](#pandas_custom_func)<br>
[Nueva columna utilizando la información de 2 columnas](#pandas_custom_func_2_cols)<br>

## Best practices sklearn
[Helpers sklearn](#helpers_sklearn)<br>
[sklearn Transformers (**CORE IDEA**)](#transformers)<br>
[sklearn Pipeline (**CORE IDEA**)](#pipeline)<br>
[ColumnTransformer (**CORE IDEA**)](#column_transformer)<br>
[Ejercicio Práctico](#practice)<br>
[Estrategias de Cross Validation](#cv_strategies)<br>
[GridSearch de un Pipeline (**BONUS**)](#gs_pipeline)<br>

## Conclusión
[Conclusión](#conclusion)<br>

## Referencias
[Referencias y lecturas recomendables](#referencias)<br>

# Imports del notebook
<a id='imports_notebook'></a>
[Volver al índice](#index)

En este apartado hacemos los principales imports del notebook.<br>
Sobre todo vamos a trabajar con **numpy**, **pandas** y **sklearn**.

In [1]:
import time

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score

Todos aquellos alumnos que trabajan en Colab, deben actualizar su versión de sklearn a 1.2 para que funcione el notebook.

En caso de no poder hacerlo, es mejor que utilicen el notebook **1_BEST_PRACTICES_PANDAS_SKLEARN**

In [2]:
# !pip install scikit-learn==1.2

![PipInstallColab](pictures/colab_sklearn_1_2.png)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, "data")

print(CWD)
print(DATA_PATH)

/Users/nicolaepopescul/code/nuclio/pandas_y_sklearn
/Users/nicolaepopescul/code/nuclio/pandas_y_sklearn/data


In [5]:
print("Working with these versions of libraries\n")
print("-"*50)
print(f"Numpy version {np.__version__}")
print(f"Pandas version {pd.__version__}")
print(f"Sklearn version {sklearn.__version__}")

Working with these versions of libraries

--------------------------------------------------
Numpy version 1.26.4
Pandas version 2.2.1
Sklearn version 1.4.1.post1


# Best practices pandas

# Helpers pandas
<a id='helpers_pandas'></a>
[Volver al índice](#index)

En este apartado definimos las principales funciones auxiliares y que serán de ayuda en el resto del notebook.

Para todos los ejercicios de **pandas**, vamos a utilizar un dataset dummy.

In [6]:
def get_category(nr_rows):
    '''
    Generates random category.
    '''
    categories = ["A", "B", "C", "X", "Y", "Z"]
    random_categories = np.random.choice(categories, size = nr_rows)
    
    return random_categories

In [7]:
def generate_dummy_dataframe(nr_rows, nr_columns, nr_categoricals):
    '''
    Generates a dummy DataFrame.
    '''
    la = []
    
    for nr_ in range(nr_columns):
        la.append(np.random.randint(-1000, 1000, size = nr_rows))
            
    df = pd.DataFrame(la).T
    
    cat_index = [cat + str(i) for i, cat in enumerate(get_category(nr_rows = nr_rows))]
    df.index = cat_index
    
    dummy_column_names = ["Column_{}".format(i) for i in range(1, nr_columns + 1)]
    
    df.columns = dummy_column_names
    
    for nr_categorical in range(nr_categoricals):
        df["Categorical_{}".format(nr_categorical + 1)] = get_category(nr_rows = nr_rows)
    
    return df
    

In [8]:
df = pd.read_csv(os.path.join(DATA_PATH, "pd_sklearn_dummy.csv"))

Veamos nuestro dataframe

In [9]:
df.head(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
0,-792,-125,51,X,Y,Z
1,-866,-397,723,Y,B,A
2,192,513,942,X,C,Y
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C


## ¿Que es el pandas groupby?
<a id='pandas_groupby'></a>
[Volver al índice](#index)<br>

La gran mayoría de los ejemplos que siguen a continuación, se van a basar en el **pandas groupby**.<br>
Por este motivo, es de vital importancia entender que hay **"dentro"** de un pandas groupby y saber cuando es el momento de usarlo.

Veamos un ejemplo práctico con nuestro dataset.

### Pregunta 1: hacer la suma de Column_1 cuando Categorical_1 == "B"

In [10]:
# code goes here

In [11]:
# posible solución
df[df["Categorical_1"] == "B"]["Column_1"].sum()

1006

Cuando queremos hacer un cálculo rápido y sencillo, podemos utilizar los **filtros de pandas** para hacer una selección de un subconjunto de datos y aplicar nuestra fórmula.

No obstante, ¿Que pasaría si necesitamos calcular esta suma para todas las categorías que hay en Categorical_1?

Una implementación rápida sería utilizar una for loop, como en el siguiente ejemplo.

In [12]:
# posible solución
result_dict = {}

for category in sorted(df["Categorical_1"].unique()):
    result_dict[category] = df[df["Categorical_1"] == category]["Column_1"].sum()
    
result_dict

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

La implementación anterior es válida, pero hay un forma mucho mejor de hacerlo. Utilizar el pandas groupby.

In [13]:
# creamos un objeto de pandas groupby y lo asignamos a gb_df
gb_df = df.groupby("Categorical_1")
gb_df

Cuando montamos un pandas groupby e inspeccionamos el objetvo, vemos que el resultado es algo similar a esto:
#### **<pandas.core.groupby.generic.DataFrameGroupBy object at 0x7fe5c81de430>**

Lo que obtenemos no es más que la dirección en memoria del pandas groupby (en nuestro caso, gb_df).

Ahora bien, una forma muy sencilla para ver el contenido del gb_df es utilizar el método **get_group**. Al método get_group le debemos pasar una tupla. El tamaño de la tupla debe ser igual al número de columnas por las cuales has hecho el groupby.

Por ejemplo:<br>

---

**Agrupamos por 2 columnas, la tupla debe ser de 2 elementos**

gb_df = df.groupby(["Cat1", "Cat2"])<br>
gb_df.get_group(**(e1, e2)**)

**Agrupamos por 3 columnas, la tupla debe ser de 3 elementos**

gb_df = df.groupby(["Cat1", "Cat2", "Cat3"])<br>
gb_df.get_group(**(e1, e2, e3)**)

Donde e1, e2 y e3 son elemento 1, elemento 2 y elemento 3 y son categorías que existen en la Columna Cat1, Cat2 y Cat3 respectivamente (e1 existe en Cat1, e2 existe en Cat2 y e3 existe en Cat3).

---

### La única condición que deben cumplir los elementos de la tupla, es ser categorías que existen dentro de las columnas del groupby.

Veamos unos ejemplos con nuestro dataframe.

In [14]:
# Hice el groupby por Categorical_1, la categoría A existe dentro de Categorical_1
# Por este motivo, cuando hago get_group(("A")) veo que todos los elementos de Categorical_1 == "A"
gb_df.get_group(("A")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [15]:
# lo mismo pero con el grupo C
gb_df.get_group(("C")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
5,59,990,-185,C,X,C
11,863,-718,736,C,X,Z
15,543,-164,-787,C,A,A
28,-32,103,-729,C,X,C
29,-825,-828,-543,C,Y,A


In [16]:
# que pasaría si intento buscar el grupo I
try:
    gb_df.get_group(("I")).head()
except:
    print('El gb_df.get_group(("I")).head() ha fallado! :(')

El gb_df.get_group(("I")).head() ha fallado! :(


¡La ejecución anterior nos ha dado un error! Intenté buscar una categoría que no existe.

### (CORE IDEA) Podemos pensar en un groupby como un conjunto de dataframes pequeños (donde las categorías siempre coinciden). Por tanto, puedo aplicar la mayoría de las transformaciones propias de un df.<br>
De hecho, hacer el get_group es idéntico a hacer un filtro de pandas groupby.

In [17]:
df[df["Categorical_1"] == "A"].head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [18]:
gb_df.get_group(("A")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [19]:
df[df["Categorical_1"] == "A"].equals(gb_df.get_group(("A"))) # comprobamos que son iguales

True

<a id='tip_1_pandas_gb_index'></a>
### Si nos fijamos de manera detenida, vemos que el index del df filtrado, es el mismo que el index del get_group.

Esta parte va a ser muy importante cuando vamos a trabajar con **pandas transform**.

De momento, nos quedamos con la idea de que hacer un filtro o get_group de groupby **me permite acceder a un subconjunto del dataframe en cuestión.**

In [20]:
print(result_dict)

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}


In [21]:
# best praxis
gb_df["Column_1"].sum().to_dict()

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

### Pandas transform (CORE IDEA)
<a id='pandas_transform'></a>
[Volver al índice](#index)<br>

### Pregunta 2: añadir una nueva columna al df que sea la suma total de Column_1 respetando cada categoría de Categorical_1

In [22]:
# code goes here

In [23]:
# posible solución
result_dict_gb = df.groupby("Categorical_1")["Column_1"].sum().to_dict()

df["New_col_1"] = df["Categorical_1"].map(result_dict_gb)

df

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,New_col_1
0,-792,-125,51,X,Y,Z,11813
1,-866,-397,723,Y,B,A,6122
2,192,513,942,X,C,Y,11813
3,73,984,-620,A,B,A,1988
4,-225,168,-50,A,B,C,1988
...,...,...,...,...,...,...,...
995,-750,397,-833,B,C,Y,1006
996,-508,917,-443,Y,A,C,6122
997,213,-781,692,X,B,C,11813
998,628,-926,-316,X,Z,X,11813


La implementación anterior utiliza una variable temporal, en nuestro caso result_dict_gb para guardar los resultados y después mappear el resultado al df.  Puede parecer una buena forma de hacerlo hasta que conocemos al pandas **transform**.

In [24]:
# best praxis
df["New_col_1_bp"] = df.groupby("Categorical_1")["Column_1"].transform(np.sum)

Pandas **transform** es una mejor solución porque permite conseguir el mismo resultado
en 1 única línea de código. No hay necesidad de guardar nada de manera temporal. Con esto conseguimos un código más limpio y elegante.

Internamente dentro de groupby lo que ocurre es:
1. Calculamos la suma de Column_1 para cada categoría.
2. Hacemos el merge con el df orignal, por el index del dataframe.

[¿Os acordáis cuando decíamos que dentro del get_group se preserva el index original?](#tip_1_pandas_gb_index)

Si nos fijamos en la siguiente línea de código, el index es idéntico al df original.

In [25]:
df.groupby("Categorical_1")["Column_1"].transform(np.sum).head()

0    11813
1     6122
2    11813
3     1988
4     1988
Name: Column_1, dtype: int64

In [26]:
df.head(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,New_col_1,New_col_1_bp
0,-792,-125,51,X,Y,Z,11813,11813
1,-866,-397,723,Y,B,A,6122,6122
2,192,513,942,X,C,Y,11813,11813
3,73,984,-620,A,B,A,1988,1988
4,-225,168,-50,A,B,C,1988,1988


### Groupby eficiente (CORE IDEA)
<a id='pandas_gbe'></a>
[Volver al índice](#index)<br>

En más de una ocasión, queremos hacer varios cálculos dentro de un pandas groupby (por ejemplo: hacer diferentes funciones de agregación como la suma, media, máximos sobre diferentes columnas) y después **concatenar** estos resultados en un nuevo dataframe. Hacemos esto a menudo durante el **EDA** en un proyecto de Machine Learning.

---
Habitualmente el patrón que se sigue es:

1. variable_1 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_1"].funcion_agregación()
2. variable_2 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_2"].otra_funcion_agregación()
3. df_resumen = pd.concat([variable_1, variable_2], axis = 1)

### Pregunta 3: crear un df_resumen que contenga la suma, media (para Column_1) y un contador de los valores positivos para la columna Column_2, agrupados por Categorical_1

El resultado que nos debería dar un df similar a este.

In [27]:
result_data_dict = {
    'Sum_column_1': {'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769},
    'Mean_column_1': {'A': 10.57, 'B': 5.78, 'C': -9.27, 'X': 78.75, 'Y': 35.59, 'Z': 67.84},
    'Counter_non_negative_values_column_2': {'A': 101, 'B': 85, 'C': 89, 'X': 77, 'Y': 91,'Z': 73}
}

In [28]:
pd.DataFrame(data = result_data_dict)

,Sum_column_1,Mean_column_1,Counter_non_negative_values_column_2
A,1988,10.57,101
B,1006,5.78,85
C,-1595,-9.27,89
X,11813,78.75,77
Y,6122,35.59,91
Z,9769,67.84,73


In [29]:
# code goes here

In [30]:
# posible solución
suma_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].sum()
mean_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].mean()
positive_counter = df.groupby("Categorical_1")["Column_2"].agg(lambda series: len(series[series >= 0]))

df_resumen = pd.concat([
    suma_column_1_categorical_1,
    mean_column_1_categorical_1,
    positive_counter
], axis = 1)

df_resumen.columns = ["Sum_column_1", "Mean_column_1", "Counter_non_negative_values_column_2"]

del suma_column_1_categorical_1, mean_column_1_categorical_1, positive_counter

df_resumen.round(2)

,Sum_column_1,Mean_column_1,Counter_non_negative_values_column_2
Categorical_1,,,
A,1988,10.57,101
B,1006,5.78,85
C,-1595,-9.27,89
X,11813,78.75,77
Y,6122,35.59,91
Z,9769,67.84,73


Si miramos de manera crítica la celda anterior, podemos ver las siguientes ineficiencias:
1. Generamos 3 variables temporales que no nos hacen falta a posteriori.
2. Debemos añadir varias líneas de código para concatenar estos resultados.
3. Tenemos que hacer limpieza de las variables temporales (buena praxis).
4. Debemos renombrar las columnas del df_resumen.

Todos los pasos anteriores para conseguir un df_resumen.

Veamos una forma mucho más elegante y eficiente de hacerlo.

In [31]:
# best praxis
df.groupby("Categorical_1").agg(
    Sum_column_1 = ("Column_1", np.sum),
    Max_column_1 = ("Column_1", np.max),
    Mean_column_1 = ("Column_1", np.mean),
    Counter_non_negative_values_column_2 = ("Column_2", lambda series: len(series[series >= 0])),
    Sum_non_negative_values_column_2 = ("Column_2", lambda series: np.sum(series[series >= 0])),
).round(2)

,Sum_column_1,Max_column_1,Mean_column_1,Counter_non_negative_values_column_2,Sum_non_negative_values_column_2
Categorical_1,,,,,
A,1988,987,10.57,101,51758
B,1006,989,5.78,85,33793
C,-1595,958,-9.27,89,43271
X,11813,995,78.75,77,35980
Y,6122,999,35.59,91,48768
Z,9769,998,67.84,73,33750


Conseguimos el mismo resultado que antes, pero de una manera más rápida y elegante. 

**En realidad, hacemos 2 operaciones adicionales y todo con menos líneas de código.**

### ¿Todos los alumnos has conseguido sacar el resultado Counter_non_negative_values_column_2?

### Funciones personalizadas
<a id='pandas_custom_func'></a>
[Volver al índice](#index)<br>

En el ejemplo anterior, para calcular **Counter_non_negative_values_column_2** tuvimos que crear una función personalizada (custom function) en forma de ***lambda***.

Esto es de extrema relevancia porque significa que dentro de un **pandas groupby** yo puedo utilizar no solo las funciones tipo: suma, media, máximo etc. sino **cualquier función siempre y cuando haga un cálculo y devuelva un resultado agregado.** De hecho, el resultado podría hasta ser un string. 

Veamos algunos ejemplos.

In [32]:
def custom_func_non_negative_sum(series):
    '''
    Sums non negative values from a pandas series
    '''
    return np.sum(series[series >= 0])

La función anterior es idéntica a la siguiente función anónima **lambda series: np.sum(series[series >= 0])**

In [33]:
df.groupby("Categorical_1")["Column_2"].agg(custom_func_non_negative_sum)

Categorical_1
A    51758
B    33793
C    43271
X    35980
Y    48768
Z    33750
Name: Column_2, dtype: int64

Por supuesto la función anterior la podemos utilizar con un pandas transform

In [34]:
df["New_col_2_cf_bp"] = df.groupby("Categorical_1")["Column_2"].transform(custom_func_non_negative_sum)

In [35]:
df.head(1)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,New_col_1,New_col_1_bp,New_col_2_cf_bp
0,-792,-125,51,X,Y,Z,11813,11813,35980


Antes hemos comentado, que la función de agregación customizada puede hasta devolver un string como resultado.

Veamos un ejemplo rápido.

In [36]:
def custom_func_returns_string(series):
    '''
    Custom function that returns a string as a result
    '''
    positive_values = len(series[series >= 0])
    negative_values = len(series[series < 0])
    
    return f"Positive Values: {positive_values} Negative Values: {negative_values}"

In [37]:
print(df.groupby("Categorical_1")["Column_1"].agg(custom_func_returns_string))

Categorical_1
A    Positive Values: 99 Negative Values: 89
B    Positive Values: 89 Negative Values: 85
C    Positive Values: 91 Negative Values: 81
X    Positive Values: 82 Negative Values: 68
Y    Positive Values: 83 Negative Values: 89
Z    Positive Values: 81 Negative Values: 63
Name: Column_1, dtype: object


In [38]:
df["New_col_2_cfs_bp"] = df.groupby("Categorical_1")["Column_1"].transform(custom_func_returns_string)

In [39]:
df.sample(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,New_col_1,New_col_1_bp,New_col_2_cf_bp,New_col_2_cfs_bp
886,248,-223,-826,C,Y,Z,-1595,-1595,43271,Positive Values: 91 Negative Values: 81
870,208,109,-517,X,Y,B,11813,11813,35980,Positive Values: 82 Negative Values: 68
120,325,72,-922,B,C,X,1006,1006,33793,Positive Values: 89 Negative Values: 85
329,52,733,-989,B,Y,A,1006,1006,33793,Positive Values: 89 Negative Values: 85
826,861,162,490,C,B,C,-1595,-1595,43271,Positive Values: 91 Negative Values: 81


En la práctica no obstante, usamos este tipo de funciones sólo cuando queremos hacer un reporting sobre un dataframe e incluirlo en un informe final.

### Nueva columna utilizando la información de 2 columnas
<a id='pandas_custom_func_2_cols'></a>
[Volver al índice](#index)<br>

Vamos a darle otra vuelta de tuerca a las funciones personalizadas.

Si hacemos un breve recap de lo visto hasta ahora, sabemos que:
1. Dentro de un groupby hay un montón de dfs "pequeños". Estos dataframes no se solapan con otros porque tienen categorías únicas (en función de las columnas por las cuales se ha hecho el groupby).
2. De alguna manera, hacer un groupby y llamar get_group (que es lo que ocurre dentro de un groupby pero para todas las categorías) es similar a hacer un filtro de df.
3. Al hacer un filtro o groupby, sigo operando todo el rato con un df. Si sigo operando con un df, puedo acceder a cualquier columna del df.
4. Por tanto, puedo realizar una función de agregación teniendo en cuenta la información de más de una columna, sin necesidad de **iterar** sobre el df.

Veamos un ejemplo sencillo:

### Pregunta 4: calcular la media de la Column_1 cuando tengo en la Column_2 más valores positivos que negativos (len_positivos >= len_negativos), en caso contrario devolver -1 todo esto dentro de un groupby.

In [40]:
# code goes here

In [41]:
# posible solución
d_ = {}

for category in sorted(df["Categorical_1"].unique()):

    sample_df = df[
        (df["Categorical_1"] == category)
    ]

    value_counts = (sample_df["Column_2"] >= 0).value_counts()
    
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        d_[category] = sample_df["Column_1"].mean()
    else:
        d_[category] = -1

d_

{'A': 10.574468085106384,
 'B': -1,
 'C': -9.273255813953488,
 'X': 78.75333333333333,
 'Y': 35.593023255813954,
 'Z': 67.84027777777777}

In [42]:
# best praxis
def custom_agg_function_2_cols(df):
    '''
    Custom function that uses values from 1 column to calculate the value for another column.
    '''
    value_counts = (df["Column_2"] >= 0).value_counts()
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        return df["Column_1"].mean()
    else:
        return -1

df.groupby("Categorical_1").apply(custom_agg_function_2_cols)

/var/folders/m8/pkh7yz5j3_318g64strn1y680000gn/T/ipykernel_6307/1531782495.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("Categorical_1").apply(custom_agg_function_2_cols)


Categorical_1
A    10.574468
B    -1.000000
C    -9.273256
X    78.753333
Y    35.593023
Z    67.840278
dtype: float64

---
# Best Practices con sklearn

# Helpers sklearn
<a id='helpers_sklearn'></a>
[Volver al índice](#index)

En este apartado definimos las principales funciones auxiliares y que serán de ayuda para la sección de sklearn.

In [43]:
'''
El DataLoader es una clase auxiliar que va a cargar nuestros datasets
Le suministramos la ruta del train/X y del test/dataset producción (prod) y nos devuelve los dos datasets
También le tenemos que pasar la columna del target y del index
Para que el objeto DataLoader distinga estas dos columnas
'''

class DataLoader(object):
    '''
    DataLoader helps you import you train and test data and do some basic preprocessing on them.
    '''
    def __init__(self, train_path, test_path, train_columns, target, index):
        '''
        Constructor for the class.
        Needs the train and test path and train_columns (features), index and target column.
        '''
        self.train_path = train_path
        self.test_path = test_path
        
        self.train_columns = train_columns
        self.target = target
        self.index = index
        
    def _process_df(self, df):
        '''
        Converts the columns to upper, sets index and splits between X and y.
        '''
        df.columns = map(str.upper, df.columns)
        df.set_index(self.index, inplace = True)
        
        if self.target in df.columns:
            
            y = df[self.target]
            df.drop(self.target, axis = 1, inplace = True)
            df = df[self.train_columns]
            
            return df, y
        
        else:
            
            df = df[self.train_columns]
            
            return df, None
    
    def load_data(self):
        '''
        Loads the data and calls _process_df to X_train and X_test.
        '''
        X_train, y_train = self._process_df(pd.read_csv(self.train_path))
        
        X_test, _ = self._process_df(pd.read_csv(self.test_path))
        
        return X_train, y_train, X_test

'''
DataFrameReporter nos va a permitir hacer nuestro primer contacto con el dataset.

Para las variables númericas hará un describe y contará los nulos que hay.
Y para las variables categóricas hará un count de los nulos y nos dirá también 
el número de categorías únicas que hay.

Al final se trata de una clase auxiliar que hará una parte del EDA.
'''

class DataFrameReporter(object):
    '''
    Helper class that reports nulls and datatypes of train and test data
    '''
    def __init__(self, X_train, X_test, target_column):
        '''
        Constructor for the class.
        Needs train and test data and also the target column in train.
        '''
        self.X_train = X_train
        self.X_test = X_test
        self.target_column = target_column
        
    def analyze_X(self, X):
        '''
        Analyses the DataFrame you pass and returns a report of nulls, distribution and other goodies.
        '''
        
        if self.target_column in X.columns:
            X = X.drop(self.target_column, axis = 1)
            
        dtypes = X.dtypes.to_frame().rename(columns = {0:"Dtypes"})

        nulls_in_train = X.isnull().sum().to_frame().rename(columns = {0:"Absolute_nulls"})
        nulls_in_train["Relative_nulls"] = nulls_in_train["Absolute_nulls"]/X.shape[0]
        nulls_in_train["Relative_nulls"] = nulls_in_train["Relative_nulls"].apply(
            lambda number: round(number, 3) * 100
        )
        
        nulls_in_train = pd.concat([nulls_in_train, dtypes], axis = 1)
        nulls_in_train["Shape"] = X.shape[0]
        nulls_in_train = nulls_in_train[["Dtypes", "Shape", "Absolute_nulls", "Relative_nulls"]]

        describe_values_num = X.describe().T
        report_df = pd.concat([nulls_in_train, describe_values_num], axis = 1)

        object_columns = X.select_dtypes("object").columns
        unique_categories = {col:X[col].nunique() for col in object_columns}
        unique_cat_df = pd.DataFrame(
            data = unique_categories.values(), 
            index = unique_categories.keys(), 
            columns = ["Unique_category"]
        )

        report_df = pd.concat([report_df, unique_cat_df], axis = 1)

        report_df.fillna("", inplace = True)
        report_df.sort_values("Dtypes", ascending = True, inplace = True)
        
        return report_df
        
    def get_reports(self):
        '''
        Calls analyze_X method and returns report DataFrame for train and test.
        '''
        report_train = self.analyze_X(X = self.X_train)
        report_test = self.analyze_X(X = self.X_test)
        
        report_train["Origin"] = "X"
        report_test["Origin"] = "Prod"
        
        result = pd.concat([report_train, report_test])
        
        return result

La siguiente línea de código es muy importante.

Hasta la [versión 1.2 de sklearn](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_set_output.html) todos los outputs de sklearn eran **numpy array**.

Utilizando la siguiente línea de código:

> set_config(transform_output = "pandas")

El output siempre será un pandas DataFrame.

In [44]:
set_config(transform_output = "pandas")

In [45]:
# Definimos nuestras variables globales
# En este caso, vamos a usar unas pocas columnas del dataframe a modo de ejemplo
# Es buena praxis definir este tipo de variables en un fichero json o bien al comienzo de un
# notebook/script

TRAIN_PATH = os.path.join(DATA_PATH, "pd_sklearn_data.csv")
PRODUCTION_PATH = os.path.join(DATA_PATH, "pd_sklearn_prod.csv")

TARGET = "SURVIVED"
INDEX = "PASSENGERID"

TRAIN_COLUMNS = ['PCLASS', 'AGE', 'SIBSP', 'EMBARKED']

# Cargamos nuestros datasets con la clase auxiliar

X, y, X_prod = DataLoader(
    train_path = TRAIN_PATH,
    test_path = PRODUCTION_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

In [46]:
# separamos nuestro dataset entre columnas numericas y object

numeric_columns = X.select_dtypes(include = np.number).columns.tolist()
object_columns = X.select_dtypes(exclude = np.number).columns.tolist()

assert (len(numeric_columns) + len(object_columns)) == X.shape[1], "You have missed some columns"

In [47]:
print("Working with numeric columns: ", ", ".join(numeric_columns))
print("Working with categorical columns: ", ", ".join(object_columns))

Working with numeric columns:  PCLASS, AGE, SIBSP
Working with categorical columns:  EMBARKED


In [48]:
# Hacemos nuestro análisis rápido de los datasets
report = DataFrameReporter(
    pd.concat([X, y], axis = 1), X_prod, TARGET
).get_reports()

In [49]:
report

,Dtypes,Shape,Absolute_nulls,Relative_nulls,count,mean,std,min,25%,50%,75%,max,Unique_category,Origin
PCLASS,int64,891,0,0.0,891.0,2.308642,0.836071,1.0,2.0,3.0,3.0,3.0,,X
SIBSP,int64,891,0,0.0,891.0,0.523008,1.102743,0.0,0.0,0.0,1.0,8.0,,X
AGE,float64,891,177,19.9,714.0,29.699118,14.526497,0.42,20.125,28.0,38.0,80.0,,X
EMBARKED,object,891,2,0.2,,,,,,,,,3.0,X
PCLASS,int64,418,0,0.0,418.0,2.26555,0.841838,1.0,1.0,3.0,3.0,3.0,,Prod
SIBSP,int64,418,0,0.0,418.0,0.447368,0.89676,0.0,0.0,0.0,1.0,8.0,,Prod
AGE,float64,418,86,20.6,332.0,30.27259,14.181209,0.17,21.0,27.0,39.0,76.0,,Prod
EMBARKED,object,418,0,0.0,,,,,,,,,3.0,Prod


### sklearn Transformers (CORE IDEA)
<a id='transformers'></a>
[Volver al índice](#index)<br>

El **sklearn Transformer** será la pieza clave dentro de la sección de best practices con sklearn.

Para comprender la utilidad del transformer, imaginemos una situación típica de un Data Scientist.

Un DS debe tratar con mucha cantidad de datos y realizar **diferentes operaciones** sobre estos. 

En un caso podría necesitar **imputar** valores nulos, en otro caso **estandarizar** un dataset, en una situación podría necesitar **convertir** datos categóricos en númericos, **reducir** la dimensionalidad, **clusterizar** los datos o bien hacer todas estas operaciones en un orden determinado.

**De hecho, lo habitual es tener que hacer diferentes operaciones sobre diferentes columnas para un mismo dataset.**

Debido a que estas operaciones son tan frecuentes y aunque tenemos a nuestra disponibilidad **pandas** y Python puro para conseguir el resultado deseado, veremos una nueva forma de hacer esta manipulación con: **pipelines y sklearn transformers**.

### Pregunta 5: Imputar el valor más frecuente para la columna de EMBARKED.

In [50]:
# code goes here

In [51]:
# posible solución
most_frequent_value = X["EMBARKED"].value_counts().index[0]
X["EMBARKED_FILLED_1"] = X["EMBARKED"].fillna(most_frequent_value)

Vamos a ver como podemos conseguir el mismo resultado con un **Transformer** de sklearn

Un **Transformer** en sklearn, es un objeto que tiene implementado el **método fit** y el **método transform**.

El alumno ya se encontró con el **método fit** muchas veces: es la llamada que hacemos cuando queremos que nuestro modelo **aprenda algo del dataset**.

En el caso de un transformer, ocurre exactamente lo mismo. Cuando llamamos el **método fit** mientras estamos utilizando un Transformer lo que le estamos diciendo es que aprenda algo del dataset (la media, la varianza etc).

**En el caso de SimpleImputer(strategy="most_frequent") el valor más frecuente que hay en un dataset o una columna.**

En el caso de StandardScaler() podría ser la media y la varianza.

### Por tanto, lo que vamos a aprender con un transformer depende del transformer con el que trabajamos y como lo instanciamos.

El **método transform** a su vez aplica lo aprendido en el **fit** al dataset y **lo transforma**.

En el ejemplo de abajo vamos a utilizarSimpleImputer(strategy="most_frequent"):

1. En el fit aprendo el valor más frecuente.
2. Con el método transform imputo el valor más frecuente a la columna en cuestión.

Vayamos por partes:

In [52]:
# instanciamos el transformer con la estrategia de imputación de "most_frequent"
# de momento nuestro transformer "no sabe nada" sobre el dataset
# si intentamos mirar cual es el valor más frecuente con
# imputer_mf.statistics_ 
# nos dará error

imputer_mf = SimpleImputer(strategy = "most_frequent")

In [53]:
try:
    imputer_mf.statistics_ 
except Exception as e:
    print(e)

'SimpleImputer' object has no attribute 'statistics_'


In [54]:
imputer_mf.fit(X[["EMBARKED"]]) # seleccion EMBARKED con doble [[]] para que sea un df

SimpleImputer(strategy='most_frequent')

In [55]:
imputer_mf.fit(X["EMBARKED"].values.reshape(-1, 1)) # o bien le paso un array de numpy al transformer

SimpleImputer(strategy='most_frequent')

In [56]:
# esto me dice que el valor más frecuente del la columna EMBARKED es S.
imputer_mf.statistics_ 

array(['S'], dtype=object)

In [57]:
# creo una nueva columna con el valor imputado
X["EMBARKED_FILLED_2"] = imputer_mf.transform(X[["EMBARKED"]])

/Users/nicolaepopescul/code/venvs/env_main/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [58]:
assert (X["EMBARKED_FILLED_1"] != X["EMBARKED_FILLED_2"]).sum() == 0

Como podemos comprobar con el assert, las dos formas de hacerlo dan exactamente el mismo resultado.
Puede parecer de momento que no ganamos nada con elegir una forma u otra de hacer las operaciones.

Ahora bien, que pasaría si tengo que hacer la misma operación sobre el dataset de test/producción. Si lo hago con pandas, debo utilizar la variable **"most_frequent_value"** para imputar los missing en el test/producción. Al tratarse de una columna, parece que no es un problema, pero todo se complica si tengo que imputar diferentes valores en diferentes columnas.

Con un **Transformer** esto es pan comido porque puedo aplicarlo a todo un dataframe y no sólo a una columna.

In [59]:
X.drop(["EMBARKED_FILLED_1", "EMBARKED_FILLED_2"], axis = 1, inplace = True)

In [60]:
imputer_df = SimpleImputer(strategy = "most_frequent")
imputer_df.fit(X)

SimpleImputer(strategy='most_frequent')

In [61]:
# En una única llamada, conseguí aprender los valores más frecuentes
# de cada una de las columnas de mi df de train.
imputer_df.statistics_

array([3, 24.0, 0, 'S'], dtype=object)

In [62]:
# ahora voy a aplicar la misma transformacion al df de test.
# muy eficiente, rápido y limpio.

X_prod_imputed = imputer_df.transform(X_prod)
X_prod_imputed.sample(10)

,PCLASS,AGE,SIBSP,EMBARKED
PASSENGERID,,,,
1140,2,29.0,1,S
991,3,33.0,0,S
1147,3,24.0,0,S
984,1,27.0,1,S
1250,3,24.0,0,Q
1182,1,24.0,0,S
1020,2,42.0,0,S
1204,3,24.0,0,S
1093,3,0.33,0,S


Lo que hemos conseguido en la celda anterior es bastante increíble. En 3 líneas de código, hemos podido crear un objeto que aprende todas las categorías más frecuente de cada columna y posteriormente aplicarlo al dataset de test.

Sigamos con nuestros ejemplos:

### sklearn Pipeline (CORE IDEA)
<a id='pipeline'></a>
[Volver al índice](#index)<br>

En el ejemplo anterior, hemos realizado 1 operación a **TODO** el df.

### ¿Que pasaría si quiero hacer primero una imputación y luego estandarizar el dataset?

Puedo aplicar 2 Transformers en secuencial, pero también puedo concatenarlos en un Pipeline.

In [63]:
pipe = Pipeline(
    steps =
    [
        ("Imputer", SimpleImputer(strategy = "mean")),
        ("Scaler", StandardScaler())   
    ]
)

El parámetro clave del Pipeline es **steps.**

Dentro de los **steps**, debemos pasar tuplas y cada tupla debe tener 2 parámetros.

### ("Nombre_del_paso", Transformer())

>Nombre_del_paso: que sea un string

>Transformer: cualquier instancia de un sklearn Transformer. Por este motivo llamamos StandardScaler() con el paréntesis.

**El funcionamiento del Pipeline es idéntico a un Transformer.**

Cuando llamo el método de fit: el pipeline pasa el dataset a cada uno de los Transformers que hay dentro y estos a su vez llaman fit y aprender diferentes estadísticos sobre el dataset.

Posteriormente, cuando llamo el método de transform, el pipeline pasa el dataset a cada uno de los Transformers y estos llaman el transform y se modifica nuestro dataset con lo aprendido en el fit.

In [64]:
# en este caso, hemos imputado los nulos y luego estandarizado el dataset.
pipe.fit_transform(X[numeric_columns])[:5]

,PCLASS,AGE,SIBSP
PASSENGERID,,,
1,0.827377,-0.592481,0.432793
2,-1.566107,0.638789,0.432793
3,0.827377,-0.284663,-0.474545
4,-1.566107,0.407926,0.432793
5,0.827377,0.407926,-0.474545


En la siguiente celda, accedemos al **"paso/step"** Scaler de nuestro pipeline.

In [65]:
pipe["Scaler"]

StandardScaler()

In [66]:
del pipe

### ColumnTransformer (CORE IDEA)
<a id='column_transformer'></a>
[Volver al índice](#index)<br>

En los ejemplos anteriores, vimos como funcionan los **sklearn Transformers y los Pipelines.**

Básicamente los Pipelines nos permiten concatencar varios pasos/transformaciones de nuestro dataset.

La idea consiste en: entra un dataset a nuestro pipeline y todos los pasos del pipelines son Transformers. Esto permite que en cada uno de los pasos, se llame al método de fit y transform de cada Transformer.

Cuando nuestro dataset ha viajado por todo el Pipeline, conseguimos un dataset Transformado
(imputado, escalado etc).

### Una cuestión pendiente que nos quedó por contestar era: ¿como podemos hacer un tratamiento específico en función de las columnas?

### Por ejemplo: imputar en 1 columna la media y en otra la mediana. O por ejemplo hacer un StandardScaler de las númericas y un OneHotEnconder las categóricas.

Para esto vamos a necesitar al primo hermano del Pipeline: **ColumnTransformer**.

In [67]:
# El ColumnTransformer tiene una sintaxis muy parecida a la del Pipeline.
# En vez de "steps" tenemos "transformers"

# Y en la tupla que pasamos (cada "step/transformer") además del nombre y el Transformer necesitamos las columnas
# con las que quieres trabajar.

# Estas las podemos pasar como una lista con los nombres de las columnas (ejemplo: numeric_columns)
# O bien como índices: mirad el ejemplo de la siguiente celda.

# Definimos nuestros ColumnTransformer de Imputación de columnas

impute_pipe = ColumnTransformer(transformers = [
    ("numeric_impute", SimpleImputer(strategy = "mean"), numeric_columns),
    ("categorical_impute", SimpleImputer(strategy = "most_frequent"), object_columns)
], remainder = "drop") 

# drop significa: aquellas columnas que no te especifico antes, eliminalas del dataset final

In [68]:
imputed_pipe_object_columns = list(map(lambda column_name: "categorical_impute__" + column_name, object_columns))
imputed_pipe_object_columns

['categorical_impute__EMBARKED']

In [69]:
# Definimos nuestros ColumnTransformer de Escalado y OneHotEncoder
# Nota: Podemos pasar una lista con los  nombres de las columnas con imputed_pipe_object_columns
# o índices de las columnas (transform_pipe) --> [0, 1, 2]

transform_pipe = ColumnTransformer(transformers = [
    ("scaler", StandardScaler(), [0, 1, 2]),
    ("encoder", OneHotEncoder(sparse_output = False), imputed_pipe_object_columns)
])

In [70]:
# Los ColumnTransformers a su vez se puede agrupar en un pipe
# Nota: si observamos con detenimiento, vemos como el último paso del pipe es un modelo.

# Esto me permite en una única llamada hacer todas las transformaciones necesarias y luego entrenar un modelo.

pipe = Pipeline(steps = [
    ("impute", impute_pipe),
    ("transform", transform_pipe),
    ("model", DecisionTreeClassifier())
])

In [71]:
pipe.fit(X, y)

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('numeric_impute',
                                                  SimpleImputer(),
                                                  ['PCLASS', 'AGE', 'SIBSP']),
                                                 ('categorical_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['EMBARKED'])])),
                ('transform',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  [0, 1, 2]),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['categorical_impute__EMBARKED'])])),
                ('model', DecisionTreeClassifier())])

### La regla de oro para prevenir el data leakage es: fitear nuestros transformers sobre el dataset de train
### y utilizar los valores aprendidos para transformar el dataset de test/producción.

---

# --------------------------- MUY IMPORTANTE ---------------------------
# ------------------------------------------------------------------------------

## FIT_TRANSFORM SOBRE X_TRAIN Y TRANSFORM SOBRE X_TEST/X_PROD

# ------------------------------------------------------------------------------

# ----------NUNCA LLAMAMOS FIT SOBRE X_TEST/X_PROD----------

---

### Si vuelvo a llamar fit al dataset de test/producción, estoy haciendo trampa porque mirando y utilizando información que no tenía disponible en el momento del train. 

In [72]:
# estoy filtrando los dos primeros pasos del pipe y hago únicamente la transformación.
pipe[:2].transform(X_prod)[:3]

,scaler__numeric_impute__PCLASS,scaler__numeric_impute__AGE,scaler__numeric_impute__SIBSP,encoder__categorical_impute__EMBARKED_C,encoder__categorical_impute__EMBARKED_Q,encoder__categorical_impute__EMBARKED_S
PASSENGERID,,,,,,
892,0.827377,0.369449,-0.474545,0.0,1.0,0.0
893,0.827377,1.331378,0.432793,0.0,0.0,1.0
894,-0.369365,2.485693,-0.474545,0.0,1.0,0.0


In [73]:
# en una única llamada, hago las mismas transformaciones del pipe y hago la predicción final
pipe.predict(X_prod)

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,

### Ejercicio Práctico
<a id='practice'></a>
[Volver al índice](#index)<br>

In [74]:
TRAIN_PATH = os.path.join(DATA_PATH, "pd_sklearn_data.csv")
PRODUCTION_PATH = os.path.join(DATA_PATH, "pd_sklearn_prod.csv")

TARGET = "SURVIVED"
INDEX = "PASSENGERID"

TRAIN_COLUMNS = ['PCLASS', 'SEX', 'AGE', 'SIBSP', 'EMBARKED', 'PARCH', 'FARE', 'NAME']

X_practice, y_practice, X_prod_practice = DataLoader(
    train_path = TRAIN_PATH,
    test_path = PRODUCTION_PATH,
    train_columns = TRAIN_COLUMNS,
    target = TARGET,
    index = INDEX
).load_data()

In [75]:
report_practice = DataFrameReporter(
    pd.concat([X_practice, y_practice], axis = 1), X_prod_practice, TARGET
).get_reports()

In [76]:
report_practice

,Dtypes,Shape,Absolute_nulls,Relative_nulls,count,mean,std,min,25%,50%,75%,max,Unique_category,Origin
PCLASS,int64,891,0,0.0,891.0,2.308642,0.836071,1.0,2.0,3.0,3.0,3.0,,X
SIBSP,int64,891,0,0.0,891.0,0.523008,1.102743,0.0,0.0,0.0,1.0,8.0,,X
PARCH,int64,891,0,0.0,891.0,0.381594,0.806057,0.0,0.0,0.0,0.0,6.0,,X
AGE,float64,891,177,19.9,714.0,29.699118,14.526497,0.42,20.125,28.0,38.0,80.0,,X
FARE,float64,891,0,0.0,891.0,32.204208,49.693429,0.0,7.9104,14.4542,31.0,512.3292,,X
SEX,object,891,0,0.0,,,,,,,,,2.0,X
EMBARKED,object,891,2,0.2,,,,,,,,,3.0,X
NAME,object,891,0,0.0,,,,,,,,,891.0,X
PCLASS,int64,418,0,0.0,418.0,2.26555,0.841838,1.0,1.0,3.0,3.0,3.0,,Prod
SIBSP,int64,418,0,0.0,418.0,0.447368,0.89676,0.0,0.0,0.0,1.0,8.0,,Prod


### Estrategias de Cross Validation
<a id='cv_strategies'></a>
[Volver al índice](#index)<br>

Hasta ahora lo más probable que el alumno ha visto una única estrategia de validación de un modelo y es utilizando un **holdout dataset**. Es decir, partiendo de un dataset de X, separo con **train_test_split** en X_train y X_test (o bien en X_val según algunos autores). Entreno mi modelo sobre el X_train y posteriormente miro el performance en X_test para decidir si el modelo es bueno o no. Si la performance es similar entre X_train y X_test significa que no tengo overfit y puedo utilizar este modelo en producción (simplificando mucho el flujo, por supuesto).

**Antes de seguir con el resto del notebook, vale la pena reflexionar sobre que es lo que exactamente buscamos en Machine Learning.**

Un Data Scientist, cuando tiene un dataset cualquiera, **en realidad quiere contestar a las siguientes preguntas:**

1. Para este dataset en concreto, ¿que algoritmo va a ajustar mejor? Por algoritmos aquí entendemos diferentes tipos de modelos. ¿Va a funcionar mejor un **DecisionTree o un XGBoost**? ¿Una **Red Neuronal o bien una Regresión Logística**?

2. Para una algoritmo determinado, ¿cuales son los mejores hiperparametros que me van a permitir sacar el máximo rendimiento? Por ejemplo: ¿es mejor un árbol con una **max_depth de 3 y min_samples de 10** o un árbol con **max_depth de 2 y criterion de gini**?

3. ¿Cuales son las mejores imputaciones y/o transformaciones que puedo hacer sobre este dataset para sacar el máximo rendimiento? ¿Es mejor imputar los nulos con **la media o sería mejor imputar un outlier**? ¿Sacaré más rendimiento utilizando un **OneHotEncoder** o es mejor utilizar un **OrdinalEncoder**?

4. ¿Como puedo estar seguro de que el resultado en X_test es representativo? **Quizás tuve (mala) suerte en el train_test_split** y los resultados que estoy sacando no se ajustan a la realidad.

Todo esto tiene un nombre en machine learning y se llama [No Free Lunch Theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem). **Es decir: a prior no sabemos que modelo, con que parámetros, con que imputaciones van a funcionar mejor para un dataset. Hay que testear diferentes posibilidades.**

Pero testearlo sólo sobre un dataset de holdout, puede no ser estadísticamente significativo. Por este motivo, vamos a ver como podemos hacerlo utilizando **Cross Validation**.

![KFold](pictures/KFold.png)

In [77]:
def get_pipe(model):
    '''
    Builds a simple Pipeline for testing.
    '''
    
    impute_pipe = ColumnTransformer(transformers = [
        ("numeric_impute", SimpleImputer(strategy = "mean"), numeric_columns),
        ("categorical_impute", SimpleImputer(strategy = "most_frequent"), object_columns)
    ], remainder = "drop")
     
    transform_pipe = ColumnTransformer(transformers = [
        ("scaler", StandardScaler(), [0, 1, 2]),
        ("encoder", OneHotEncoder(sparse_output = False), [3])
    ])
    
    pipe = Pipeline(steps = [
        ("impute", impute_pipe),
        ("transform", transform_pipe),
        ("model", model)
    ])
    
    return pipe

In [78]:
try:
    del pipe
except:
    pass

In [79]:
pipe = get_pipe(model = DecisionTreeClassifier())

En la siguiente celda de código, voy a crear una instancia de KFold.

Voy a iterar 5 veces sobre el dataset, seleccionando en cada iteración: 80% del dataset para el train (train_idx) y 20% para validación (val_idx).

Posteriormente, voy a entrenar un **PIPE NUEVO** sobre el 80% y valido el score sobre el 20%.

Cuando acaba la loop, tengo 5 scores y puedo promediarlos para ver el resultado que me sale.

En nuestro caso aproximadamente 0,79 accuracy.

### (CORE IDEA): es muy importante cuando hacemos la cross validation, entrenar un modelo desde cero en cada iteracción. En caso contrario, le estamos mostrando a nuestro algoritmo cada vez más datos y el score puede ser demasiado optimista.

In [80]:
kfold = KFold(shuffle = True) # por defecto shuffle es False

LIST_SCORES_TRAIN = []
LIST_SCORES_VAL = []

for train_idx, val_idx in kfold.split(X, y):
    
    X_train_ = X.iloc[train_idx]
    y_train_ = y.iloc[train_idx]
    
    X_val_ = X.iloc[val_idx]
    y_val_ = y.iloc[val_idx]
    
    pipe = get_pipe(model = DecisionTreeClassifier(max_depth = 3)) # nuevo modelo en cada iteracción
    pipe.fit(X_train_, y_train_)
    
    pred_train_ = pipe.predict(X_train_)
    pred_val_ = pipe.predict(X_val_)
    
    score_train_ = accuracy_score(y_true = y_train_, y_pred = pred_train_)
    score_val_ = accuracy_score(y_true = y_val_, y_pred = pred_val_)
    
    LIST_SCORES_TRAIN.append(score_train_)
    LIST_SCORES_VAL.append(score_val_)

Vemos que en algunos casos, el score que obtengo es muy bajo y en otros muy alto. Todo esto tiene que ver con el split que hago de mi dataset en cada iteración.

In [81]:
LIST_SCORES_TRAIN

[0.7176966292134831,
 0.7152875175315568,
 0.699859747545582,
 0.7068723702664796,
 0.7040673211781207]

In [82]:
np.mean(LIST_SCORES_TRAIN)

0.7087567171470445

In [83]:
LIST_SCORES_VAL

[0.7094972067039106,
 0.6404494382022472,
 0.6235955056179775,
 0.7528089887640449,
 0.6853932584269663]

In [84]:
np.mean(LIST_SCORES_VAL)

0.6823488795430294

### GridSearch de un Pipeline (BONUS)
<a id='gs_pipeline'></a>
[Volver al índice](#index)<br>

En la sección anterior, hemos visto como podemos hacer manualmente una cross validation (CV).

Pero hemos probado siempre el mismo modelo, con los mismos transformers y las mismas imputaciones.

**Básicamente, el apartado anterior sólo me ayuda a resolver la pregunta 4/4.**

### Vamos a ver como podemos utilizar sklearn para probar diferentes imputaciones y diferentes hiperparametros para un modelo dentro de un CV. Nos ayuda a contestar las preguntas 1 y 3.

In [85]:
pipe = get_pipe(model = DecisionTreeClassifier())
pipe

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('numeric_impute',
                                                  SimpleImputer(),
                                                  ['PCLASS', 'AGE', 'SIBSP']),
                                                 ('categorical_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['EMBARKED'])])),
                ('transform',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  [0, 1, 2]),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [3])])),
                ('model', DecisionTreeClassifier())])

In [86]:
# Para hacer esta búsqueda lo más eficiente posible, debemos definir un espacio de parámetros a explorar.

# Para ello sklearn nos exige definir un diccionar que será nuestro param_grid.

# Diccionario con los parámetros de nuestro pipeline.

param_grid = {
    "impute__numeric_impute__strategy": [
        "mean", 
        "median"
    ],
    "model__criterion": ["gini", "entropy"],
    "model__max_depth": np.arange(2, 7),
    "model__min_samples_split": [3, 5, 10, 15, 20]
}

# ¿Como podemos leer esta key del diccionario param_grid?

# impute__numeric_impute__strategy

# impute: 
#    es el nombre del primer paso de nuestra pipeline --> ("impute", impute_pipe),

# numeric_impute: 
#    es el nombre de que le dimos dentro de nuestra impute_pipe al primer paso 
#    --> ("numeric_impute", SimpleImputer(strategy = "mean"), numeric_columns),

# strategy:
#    es uno de los posibles parámetros que puede tomar SimpleImputer

# Por tanto, lo que vamos a hacer a continuación es ir pasando a nuestro SimpleImputer diferentes
# estrategias de imputación: mean, median y valor más frecuente y a la vez le vamos a 
# pasar a nuestro paso "model" del pipeline, es decir, a nuestro DecisionTreeClassifier diferentes valores para
# los parámetros de criterion y max_depth y min_samples_split.

# De esta manera, buscaremos cúal es la mejor estrategia de imputación/transformación
# y los mejores hiperparámetros del modelo

In [87]:
cv = RepeatedKFold()

In [88]:
# La búsqueda de nuestro mejor modelo lo vamos a hacer con RandomizedSearchCV que explora sólo algunas 
# combinaciones de nuestro param_grid y también con GridSearchCV que explora todas las posibilidades.

st = time.time()

rand_cv = RandomizedSearchCV(
    estimator = pipe, 
    param_distributions = param_grid,
    cv = cv,
    return_train_score = True,
    verbose = False,
    n_jobs = -1
)

rand_cv.fit(X, y)
score_rand = rand_cv.score(X, y)

et = time.time()

tt_rand = round((et - st)/60, 2)
print("Total time took: ", str(tt_rand), " minutes!")

Total time took:  0.04  minutes!


In [89]:
st = time.time()

grid_cv = GridSearchCV(
    estimator = pipe, 
    param_grid = param_grid,
    cv = cv,
    return_train_score = True,
    verbose = False,
    n_jobs = -1
)

grid_cv.fit(X, y)
score_grid = grid_cv.score(X, y)

et = time.time()
tt_grid = round((et - st)/60, 2)
print("Total time took: ", str(tt_grid), " minutes!")

Total time took:  0.17  minutes!


In [90]:
pd.concat(
    [
        pd.DataFrame(
            data = list(rand_cv.best_params_.values()) + [round(score_rand, 3)], 
            index = list(rand_cv.best_params_.keys()) + ["Score"],
            columns = ["RandomizedSearchCV"]
        ),
        
        pd.DataFrame(
            data = list(grid_cv.best_params_.values()) + [round(score_grid, 3)], 
            index = list(grid_cv.best_params_.keys()) + ["Score"],
            columns = ["GridSearchCV"]
        ),
    ],
    axis = 1
)

,RandomizedSearchCV,GridSearchCV
model__min_samples_split,15,10
model__max_depth,6,5
model__criterion,entropy,gini
impute__numeric_impute__strategy,mean,median
Score,0.745,0.734


In [91]:
# Como podemos ver, en gridsearch es varias veces más lento que el RandomizedSearchCV
round((tt_grid/tt_rand), 2)

4.25

In [92]:
# elegimos nuestro mejor estimador en función del score que han sacado

final_pipe = rand_cv if score_rand >= score_grid else grid_cv

In [93]:
final_pipe.best_estimator_

Pipeline(steps=[('impute',
                 ColumnTransformer(transformers=[('numeric_impute',
                                                  SimpleImputer(),
                                                  ['PCLASS', 'AGE', 'SIBSP']),
                                                 ('categorical_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['EMBARKED'])])),
                ('transform',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  [0, 1, 2]),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [3])])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=6,
                                        min_samples_split=15))])

In [94]:
final_prediction = final_pipe.predict(X_prod)

### Conclusión
<a id='conclusion'></a>
[Volver al índice](#index)<br>

A lo largo de este notebook hemos aprendido nuevas técnicas y algunas de las best practices de **pandas** y **sklearn**.

### Siempre debemos intentar escribir un código legible, limpio y procurar reutilizar los módulos o paquetes ya disponibles (para no reinventar la rueda).

Tanto **pandas como sklearn** ofrecen mucha flexibilidad así como un montón de funcionalidades para nuestro día a día.

La utilización eficiente de pandas groupby puede ayudar mucho a la hora de hacer un análisis más rápido y conciso.

A su vez sklearn con sus Transformers, Pipelines y Cross Validation nos da herramientas muy potentes para entrenar nuestros modelos de ML.

Recomendamos a los alumnos que los vayan incorporando en su kit diario cuanto antes.

# El alumno puede probar montar un nuevo pipeline con todas las columnas del dataset para practicar lo aprendido.

### Referencias y lecturas recomendables
<a id='referencias'></a>
[Volver al índice](#index)<br>

A continuación dejamos algunos links útiles para profundizar en algunos de los conceptos que hemos visto en el notebook:

[Sitio oficial de pandas](https://pandas.pydata.org/)

[Apply vs Transform en pandas groupby](https://stackoverflow.com/questions/27517425/apply-vs-transform-on-a-group-object/47143056#47143056)

[Sitio oficial de sklearn](https://scikit-learn.org/stable/)

[Diseño de sklearn](https://arxiv.org/pdf/1309.0238.pdf)

[Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning (MUY RECOMENDABLE)
](https://arxiv.org/pdf/1811.12808.pdf)

[Diferentes estrategias de Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html)

[Duck Typing](https://en.wikipedia.org/wiki/Duck_typing)